In [1]:
import numpy as np
y = np.sin(np.pi/2)
print(y)

1.0


# Tugas Modul 3

## Metode Numerik Pencari Akar Persamaan

**SOAL 1**

Dalam Jupyter Notebook ini, kita akan membahas dua metode numerik populer untuk mencari akar-akar persamaan: **Metode Regula Falsi (False Position)** dan **Metode Secant**. Untuk setiap metode, kita akan mempelajari motivasinya, langkah-langkah algoritmanya, implementasi dalam Python, serta contoh penerapannya pada kasus fisika yang berbeda dari modul.

Per kelompok akan mempresentasikan di kelas di awal pertemuan pada 20 Maret 2025 sebelum masuk pembahasan materi berikutnya oleh dosen pengampu.

---

## 1. Metode Regula Falsi (False Position)

### Pendahuluan/Motivasi

**Metode Regula Falsi**, atau dikenal juga sebagai **Metode Posisi Palsu**, adalah salah satu metode pencari akar **tertutup**. Ini berarti metode ini membutuhkan dua titik awal, $a$ dan $b$, yang harus mengapit akar (yaitu, $f(a)$ dan $f(b)$ memiliki tanda yang berlawanan). Dengan demikian, keberadaan akar di antara $a$ dan $b$ sudah terjamin.

Motivasi utama di balik Regula Falsi adalah untuk **mempercepat konvergensi** dibandingkan dengan metode bagi dua (bisection). Alih-alih hanya membagi dua interval secara arbitrer, Regula Falsi memanfaatkan informasi nilai fungsi. Metode ini menggambar garis lurus (garis sekan) yang menghubungkan titik $(a, f(a))$ dan $(b, f(b))$. Titik potong garis sekan ini dengan sumbu $x$ kemudian dijadikan perkiraan akar yang baru. Pendekatan ini seringkali lebih efisien karena secara cerdas "mendekati" akar dengan mempertimbangkan kemiringan fungsi.

### Langkah-langkah Algoritma disertai *Pseudocode*

**Asumsi:** Fungsi $f(x)$ kontinu pada interval $[a, b]$ dan $f(a) \cdot f(b) < 0$.

1.  **Inisialisasi:**
    * Pilih dua titik awal $a$ dan $b$ sehingga $f(a) \cdot f(b) < 0$.
    * Tentukan **toleransi kesalahan** ($\epsilon$) dan **jumlah iterasi maksimum** ($max\_iter$).
2.  **Iterasi:** Lakukan perulangan hingga kriteria konvergensi terpenuhi atau jumlah iterasi maksimum tercapai.
    * Hitung titik perkiraan akar baru, $c$, menggunakan rumus interpolasi linier:
        $$c = b - \frac{f(b)(b - a)}{f(b) - f(a)}$$
    * Evaluasi $f(c)$.
    * **Perbarui Interval:**
        * Jika $f(c) = 0$, maka $c$ adalah akar eksak. Hentikan iterasi.
        * Jika $f(a) \cdot f(c) < 0$, berarti akar berada di interval $[a, c]$. Set $b = c$.
        * Jika $f(b) \cdot f(c) < 0$, berarti akar berada di interval $[c, b]$. Set $a = c$.
    * Periksa kriteria konvergensi. Misalnya, $|f(c)| < \epsilon$ atau nilai relatif perubahan akar $|(c_{baru} - c_{lama}) / c_{baru}| < \epsilon$.
3.  **Output:** Akar yang ditemukan atau pesan bahwa metode gagal konvergen dalam jumlah iterasi maksimum.

**Pseudocode:**

```pseudocode
FUNCTION RegulaFalsi(f, a, b, epsilon, max_iter):
    IF f(a) * f(b) >= 0 THEN
        PRINT "Error: f(a) dan f(b) harus memiliki tanda yang berlawanan."
        RETURN NULL
    END IF

    iter_count = 0
    c_prev = NULL  // Untuk melacak akar sebelumnya dan memeriksa konvergensi

    WHILE iter_count < max_iter:
        // Hitung perkiraan akar baru menggunakan rumus Regula Falsi
        c = b - (f(b) * (b - a)) / (f(b) - f(a))

        // Kriteria penghentian berdasarkan perubahan relatif akar
        IF c_prev IS NOT NULL AND ABS((c - c_prev) / c) < epsilon THEN
            RETURN c
        END IF

        // Kriteria penghentian berdasarkan nilai fungsi mendekati nol
        IF ABS(f(c)) < epsilon THEN
            RETURN c
        END IF

        // Perbarui interval
        IF f(a) * f(c) < 0 THEN
            b = c
        ELSE
            a = c
        END IF

        c_prev = c // Simpan akar saat ini untuk iterasi berikutnya
        iter_count = iter_count + 1

    PRINT "Metode gagal konvergen dalam jumlah iterasi maksimum."
    RETURN NULL
END FUNCTION

In [8]:
def percepatan_sudut(theta):
    g = 9.81  
    L = 1.0   
    k = 0.1   
    return (g / L) * np.sin(theta) + k * (theta**2)

a = -1.0
b = 1.0

print(f"Mencari akar fungsi percepatan_sudut(theta) = g/L * sin(theta) + k * theta^2 = 0 menggunakan Regula Falsi:")
akar_regula_falsi_fisika = regula_falsi(percepatan_sudut, a, b, epsilon=1e-7, max_iter=200)

if akar_regula_falsi_fisika is not None:
    print(f"\nSudut theta yang ditemukan saat percepatan sudut nol: {akar_regula_falsi_fisika:.6f} radian")
    print(f"Nilai fungsi pada akar: f({akar_regula_falsi_fisika:.6f}) = {percepatan_sudut(akar_regula_falsi_fisika):.6e}")

Mencari akar fungsi percepatan_sudut(theta) = g/L * sin(theta) + k * theta^2 = 0 menggunakan Regula Falsi:
Konvergen setelah 4 iterasi.

Sudut theta yang ditemukan saat percepatan sudut nol: -0.000000 radian
Nilai fungsi pada akar: f(-0.000000) = -5.998513e-11


## 2. Metode Secant

### Pendahuluan/Motivasi

**Metode Secant** adalah metode pencari akar **terbuka** yang merupakan modifikasi dari metode Newton-Raphson. Perbedaan utamanya yang signifikan adalah bahwa Metode Secant **tidak memerlukan perhitungan turunan fungsi ($f'(x)$)**. Ini adalah keuntungan besar, terutama ketika turunan fungsi sulit atau bahkan tidak mungkin dihitung secara analitis.

Motivasi di balik metode Secant adalah untuk mengaproksimasi turunan $f'(x_n)$ dalam rumus Newton-Raphson menggunakan beda hingga. Daripada menggunakan turunan eksak, metode Secant menggunakan kemiringan garis sekan yang menghubungkan dua titik perkiraan terakhir, $(x_{n-1}, f(x_{n-1}))$ dan $(x_n, f(x_n))$. Ini memungkinkan metode ini untuk beroperasi hanya dengan evaluasi fungsi. Namun, sebagai metode terbuka, **tidak ada jaminan konvergensi** untuk semua kasus atau pemilihan titik awal yang sembarang.

### Langkah-langkah Algoritma disertai *Pseudocode*

1.  **Inisialisasi:**
    * Pilih dua titik awal $x_0$ dan $x_1$. Berbeda dengan Regula Falsi, $f(x_0)$ dan $f(x_1)$ tidak harus memiliki tanda yang berlawanan.
    * Tentukan **toleransi kesalahan** ($\epsilon$) dan **jumlah iterasi maksimum** ($max\_iter$).
2.  **Iterasi:** Lakukan perulangan hingga kriteria konvergensi terpenuhi atau jumlah iterasi maksimum tercapai.
    * Hitung perkiraan akar baru, $x_{i+1}$, menggunakan rumus:
        $$x_{i+1} = x_i - f(x_i) \frac{x_i - x_{i-1}}{f(x_i) - f(x_{i-1})}$$
    * Perbarui nilai: $x_{i-1} = x_i$ dan $x_i = x_{i+1}$.
    * Periksa kriteria konvergensi, misalnya nilai relatif perubahan akar $|(x_{i+1} - x_i) / x_{i+1}| < \epsilon$.
3.  **Output:** Akar yang ditemukan atau pesan bahwa metode gagal konvergen dalam jumlah iterasi maksimum.

**Pseudocode:**

```pseudocode
FUNCTION Secant(f, x0, x1, epsilon, max_iter):
    // Cek awal untuk menghindari pembagian dengan nol jika f(x0) dan f(x1) sangat dekat
    IF ABS(f(x1) - f(x0)) < 1e-10 THEN
        PRINT "Error: f(x0) dan f(x1) terlalu dekat atau sama, metode mungkin tidak konvergen."
        RETURN NULL
    END IF

    iter_count = 0

    WHILE iter_count < max_iter:
        // Hindari pembagian dengan nol atau nilai yang sangat kecil
        IF ABS(f(x1) - f(x0)) < 1e-10 THEN
            PRINT "Penyebut mendekati nol pada iterasi ini. Metode mungkin tidak konvergen."
            RETURN NULL
        END IF

        // Hitung perkiraan akar baru menggunakan rumus Secant
        x_new = x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0))

        // Kriteria penghentian: perubahan relatif akar kurang dari epsilon
        IF ABS((x_new - x1) / x_new) < epsilon THEN
            RETURN x_new
        END IF

        // Perbarui nilai x0 dan x1 untuk iterasi berikutnya
        x0 = x1
        x1 = x_new
        iter_count = iter_count + 1

    PRINT "Metode gagal konvergen dalam jumlah iterasi maksimum."
    RETURN NULL
END FUNCTION

In [12]:
import numpy as np

# Definisikan fungsi percepatan sudut
def percepatan_sudut(theta):
    g = 9.81
    L = 1.0
    k = 0.1
    return (g / L) * np.sin(theta) + k * (theta**2)

x0 = -0.8
x1 = -0.5

print(f"\nMencari akar fungsi percepatan_sudut(theta) = g/L * sin(theta) + k * theta^2 = 0 menggunakan Metode Secant:")
akar_secant_fisika = secant_method(percepatan_sudut, x0, x1, epsilon=1e-7, max_iter=200)

if akar_secant_fisika is not None:
    print(f"\nSudut theta yang ditemukan saat percepatan sudut nol: {akar_secant_fisika:.6f} radian")
    print(f"Nilai fungsi pada akar: f({akar_secant_fisika:.6f}) = {percepatan_sudut(akar_secant_fisika):.6e}")


Mencari akar fungsi percepatan_sudut(theta) = g/L * sin(theta) + k * theta^2 = 0 menggunakan Metode Secant:
Penyebut mendekati nol pada iterasi 7. Metode mungkin tidak konvergen.
